In [15]:
class Point:
    def __init__(self, a, b):
        self.x = a
        self.y = b
        
    def __lt__(self, other):
        return self.x < other.x
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    
def sub(sel, v):
    return Point(sel.x-v.x, sel.y-v.y)
   
def add(sel, v):
    return Point(sel.x+v.x, sel.y+v.y)
   
def dot(sel, v):
    return sel.x*v.x + sel.y * v.y
   
def cross(sel, v):
    return sel.x*v.y - sel.y * v.x

def colinear(sel, v):
    return  cross(sel, v) == 0
   
def points_in_different_sides(x1, x2, a, b):
    ab = sub(b, a)
    ax1 = sub(x1, a)
    ax2 = sub(x2, b)
    return cross(ab, ax1) * cross(ab, ax2) < 0
     
def point_on_segment(x, a, b):
    if a == b:
        return a == x
    ab = sub(b, a)
    ba = sub(a, b)
    ax = sub(x, a)
    bx = sub(x, b)
    return colinear(ab, ax) and dot(ab, ax) >= 0 and dot(ba, bx) >= 0

def segments_cross(a, b, c, d):
    f1 = points_in_different_sides(c, d, a, b) and points_in_different_sides(a, b, c, d)
    f2 = point_on_segment(b, c, d) or  point_on_segment(a, c, d) or  point_on_segment(c, a, b) or  point_on_segment(d, a, b)
    return f1 or f2

def segments_cross_polygon(a, b, c, d): #для многоугольника не учитываем общую вершину
    f1 = points_in_different_sides(c, d, a, b) and points_in_different_sides(a, b, c, d)
    f2 = point_on_segment(a, c, d) or  point_on_segment(d, a, b)
    return f1 or f2

def intersect(a, b):
    return(segments_cross(a.p, a.q, b.p, b.q))

class Seg:
    def __init__(self, a, b, id):
        self.p = a
        self.q = b
        self.id = id
        
    def get_y(self, x):
        if self.p.x == self.q.x:
            return self.p.y
        else:
            return self.p.y + (self.q.y - self.p.y) * (x - self.p.x) / (self.q.x -self.p.x)
        
    def __lt__(self, b):
        x = max(min(self.p.x, self.q.x), min(b.p.x, b.q.x))
        if self.get_y(x) == b.get_y(x):
            return self.id > b.id
        return self.get_y(x) < b.get_y(x)
    
    def __eq__(self, other):
        return self.p == other.p and self.q == other.q and self.id == other.id
    
    def __hash__(self):
        return hash((hash((self.p, self.q)), self.id))
    
    
class Evnt:
    def __init__(self, x, tp, d):
        self.x = x
        self.tp = tp
        self.id = d
        
    def __lt__(self, b):
        if b.x != self.x:
            return self.x < b.x
        else:
            return self.tp > b.tp


from sortedcontainers import SortedSet
import bisect



def solution(a): #проверка многоугольника на самопересечения -- nlogn
    n = len(a)
    e = []
    for i in range(0, n):
        e.append(Evnt(min(a[i].p.x, a[i].q.x), 1, i))
        e.append(Evnt(max(a[i].p.x, a[i].q.x), -1, i))
    e.sort()
       
    s = SortedSet()
    for i in range(0, len(e)):
        id = e[i].id
        if e[i].tp == 1:
            nxt = bisect.bisect_left(s, a[id])
            prev = len(s)
            if (nxt != 0):
                prev = nxt - 1
            if nxt != len(s) and intersect(a[id], s[nxt]) and not(abs(s[nxt].id - id) <= 1 or abs(s[nxt].id - id) == n -1):
                return True
            if nxt + 1 < len(s) and intersect(a[id], s[nxt + 1]) and not(abs(s[nxt + 1].id - id) <= 1 or abs(s[nxt + 1].id - id) == n -1):
                return True
            if prev != len(s) and prev >= 0 and intersect(a[id], s[prev]) and not (abs(s[prev].id - id) <= 1 or abs(s[prev].id - id) == n -1):
                return True
            if prev != len(s) and prev - 1 >= 0 and intersect(a[id], s[prev - 1]) and not (abs(s[prev - 1].id - id) <= 1 or abs(s[prev - 1].id - id) == n -1):
                return True
            s.add(a[id])
        else:
            cur = len(s)
            nxt = bisect.bisect(s, a[id])            
            if nxt >= len(s):
                nxt = max(len(s) - 1, 0)
            while nxt > 0 and s[nxt] != a[id]:
                nxt -= 1
                
            
            cur = nxt
            nxt = cur + 1
            prev = cur - 1
            if nxt < len(s) and prev >= 0 and intersect(s[nxt], s[prev]) and not (abs(s[prev].id - s[nxt].id) <= 1 or abs(s[prev].id - s[nxt].id) == n -1):
                return True
            if nxt + 1 < len(s) and prev - 1 >= 0 and intersect(s[nxt + 1], s[prev - 1]) and not (abs(s[prev - 1].id - s[nxt + 1].id) <= 1 or abs(s[prev - 1].id - s[nxt + 1].id) == n -1):
                return True
            if nxt + 1 < len(s) and prev >= 0 and intersect(s[nxt + 1], s[prev]) and not (abs(s[prev].id - s[nxt + 1].id) <= 1 or abs(s[prev].id - s[nxt + 1].id) == n -1):
                return True
            if nxt < len(s) and prev - 1 >= 0 and intersect(s[nxt], s[prev - 1]) and not (abs(s[prev - 1].id - s[nxt].id) <= 1 or abs(s[prev - 1].id - s[nxt].id) == n -1):
                return True
            if (cur < len(s) and cur >= 0):
                s.pop(cur)
    return False


def solution_polys(a, b): #проверка простых многоугольников на пересечение
    n = len(a)
    e = []
    for i in range(0, n):
        e.append(Evnt(min(a[i].p.x, a[i].q.x), 1, i))
        e.append(Evnt(max(a[i].p.x, a[i].q.x), -1, i))
        
    for i in range(0, len(b)):
        e.append(Evnt(min(b[i].p.x, b[i].q.x), 2, i + n))
        e.append(Evnt(max(b[i].p.x, b[i].q.x), -2, i + n))
        a.append(b[i])
    
    e.sort()
    s = SortedSet()
    for i in range(0, len(e)):
        id = e[i].id
        if e[i].tp >= 1:
            nxt = bisect.bisect_left(s, a[id])
            prev = len(s)
            if (nxt != 0):
                prev = nxt - 1
            if nxt != len(s):
                if s[nxt].id < n:
                    tp_next = 1
                else:
                    tp_next = 2
                    
            if id < n:
                    tp = 1
            else:
                    tp = 2
                    
            if nxt != len(s) and intersect(a[id], s[nxt]) and not(tp == tp_next):
                return True
            
            if nxt + 1 < len(s) and nxt + 1 >= 0:
                if s[nxt + 1].id < n:
                    tp_next = 1
                else:
                    tp_next = 2
                    
            if nxt + 1 < len(s) and nxt + 1 >= 0 and intersect(a[id], s[nxt + 1]) and not(tp == tp_next):
                return True
            
            
            if prev != len(s) and prev - 1 >= 0:
                if s[prev - 1].id < n:
                    tp_prev = 1
                else:
                    tp_prev = 2
            if prev != len(s) and prev - 1 >= 0 and intersect(a[id], s[prev - 1]) and not(tp == tp_prev):
                return True
            
            s.add(a[id])
        else:
            cur = len(s)
            nxt = bisect.bisect(s, a[id])            
            if nxt >= len(s):
                nxt = max(len(s) - 1, 0)
            while nxt > 0 and s[nxt] != a[id]:
                nxt -= 1
            cur = nxt
            nxt = cur + 1
            prev = cur - 1
            
            if prev != len(s) and prev >= 0:
                if s[prev].id < n:
                    tp_prev = 1
                else:
                    tp_prev = 2
                    
            if nxt != len(s) and nxt >= 0:
                if s[nxt].id < n:
                    tp_next = 1
                else:
                    tp_next = 2
                        
            if prev != len(s) and prev - 1 >= 0:
                if s[prev - 1].id < n:
                    tp_prev1 = 1
                else:
                    tp_prev1 = 2
                    
            if nxt >= 0 and nxt + 1 < len(s):
                if s[nxt + 1].id < n:
                    tp_next1 = 1
                else:
                    tp_next1 = 2
                    
            if  nxt < len(s) and prev >= 0 and intersect(s[nxt], s[prev]) and not(tp_prev == tp_next):
                return True
            
            if  nxt < len(s) and prev >= 0  and nxt + 1 < len(s) and intersect(s[nxt + 1], s[prev]) and not(tp_prev == tp_next1):
                return True
            
            if  nxt < len(s) and prev >= 0  and prev - 1 >= 0  and intersect(s[nxt], s[prev - 1]) and not(tp_prev1 == tp_next):
                return True
            
            if  nxt < len(s) and prev >= 0  and prev - 1 >= 0 and nxt + 1 < len(s) and nxt >= 0 and intersect(s[nxt + 1], s[prev - 1]) and not(tp_prev1 == tp_next1):
                return True
            
            
            
            
            if (cur < len(s) and cur >= 0):
                s.pop(cur)
    return False        

import math

def dlina(a):
    return math.sqrt(a.x*a.x + a.y*a.y)


   
def is_simple(nums, n):  #(перебор за O(n^2))
    if (n <= 1):
        return True
    for i in range(n - 2):
        if segments_cross_polygon(nums[i], nums[i+1], nums[i+1], nums[i+2]):
            return False
        for j in range(i + 2, n - 1):
            if segments_cross(nums[i], nums[i+1], nums[j], nums[j+1]):
                return False
        if i == 0 and segments_cross_polygon(nums[i+1], nums[i], nums[i], nums[n-1]):
            return False
        if i != 0 and segments_cross(nums[i+1], nums[i], nums[0], nums[n-1]):
            return False
    return True


def is_convex(nums, n):
    if (n < 3):
        return True
    
    p = []
    for i in range(0, len(nums) - 1):
        p.append(Seg(nums[i], nums[i+1], i))

    p.append(Seg(nums[0], nums[len(nums)-1], len(nums) - 1))

    if solution(p):
        return False    
    else:
        sign = cross(sub(nums[0], nums[n-1]), sub(nums[0], nums[1]))
        for i in range(1, n - 1):
            if sign *cross(sub(nums[i],nums[i-1]), sub(nums[i], nums[1 + i])) < 0:
                return False
        if sign *cross(sub(nums[n-1], nums[n-2]), sub(nums[n-1], nums[0])) < 0:
                return False
    return True

def perim(nums, n):
    res = 0
    for i in range(0, n-1):
        res += dlina(sub(nums[i+1], nums[i]))
    res += dlina(sub(nums[0], nums[n-1]))
    return res



def square(nums, n): #площадь простого многоугольника
    s = 0
    for i in range(1, n - 1):
        s += cross(sub(nums[i], nums[0]), sub(nums[i+1], nums[0])) / 2
    return abs(s)

def colinear_segm_cross(a, b, otr1, otr2): #проверка 2 параллельных пересекющихся отрезков на кол-во общих точек(1 или беск)
    if (point_on_segment(a, otr1, otr2) and point_on_segment(b, otr1, otr2)) or (point_on_segment(otr1, a, b) and point_on_segment(otr2, a, b)):
         return 1000
    if (a == otr1 and not segments_cross_polygon(b, a, otr1, otr2)) or (b == otr1 and not segments_cross_polygon(a, b, otr1, otr2)):
        return 1
    if (a == otr2 and not segments_cross_polygon(b, a, otr2, otr1)) or (b == otr2 and not segments_cross_polygon(a, b, otr2, otr1)):
        return 1
    return 1000

def tria(a, b, c, otr1, otr2):  #количество точек пересечения треугольника и отрезка
    cnt = 0

    if segments_cross(a, b, otr1, otr2):
        if colinear(sub(a, b), sub(otr1, otr2)):
            return colinear_segm_cross(a, b, otr1, otr2)
        else:
            cnt += 1
                
    if segments_cross(b, c, otr1, otr2):
        if colinear(sub(c, b), sub(otr1, otr2)):
            return colinear_segm_cross(c, b, otr1, otr2)
        elif (point_on_segment(b, otr1, otr2)):
            cnt += 0
        else:
            cnt += 1

    if segments_cross(a, c, otr1, otr2):
        if colinear(sub(c, a), sub(otr1, otr2)):
            return colinear_segm_cross(c, a, otr1, otr2) 
        elif point_on_segment(c, otr1, otr2) or point_on_segment(a, otr1, otr2):
            cnt += 0
        else:
            cnt += 1
    return cnt  

def point_in_poly1(v, z):#для простого мнооугольника
    points = v
    n  = len(points)
    points.append(v[0])
    for i in range(n):
        a = points[i]
        b = points[i+1]
        ab = sub(b, a)
        ba = sub(a, b)
        az = sub(z, a)
        bz = sub(z, b)
        if cross(ab, az) == 0 and dot(ab, az) >= 0 and dot(ba, bz) >=0:
            return True
    ang  = 0
    for i in range(n):
        a = sub(points[i], z)
        b = sub(points[i + 1], z)
        ang += math.atan2(cross(a, b), dot(a, b))
    if abs(abs(ang) - 2 * math.pi) < 0.00001:
        return True
    return False

def point_in_poly(v, z):#для простого мнооугольника
    points = v
    ans = False
    n  = len(points)
    for i in range(n):
        if point_on_segment(z, v[i], v[i-1]):
            return True
        if (v[i].y == v[i - 1].y):
            continue
        if (z.y == min(v[i].y, v[i-1].y)):
            continue
        if (z.y == max(v[i].y, v[i-1].y) and z.x > min(v[i].x, v[i-1].x)):
            ans = not ans
        elif (((v[i].y  <= z.y and z.y < v[i-1].y) or (v[i - 1].y <= z.y and z.y < v[i].y)) and (z.x > (v[i-1].x - v[i].x) * (z.y - v[i].y) /(v[i-1].y - v[i].y) + v[i].x)):
            ans = not ans
    return ans

def circles(x1, y1, r1, x2, y2, r2): #определение точек пересечения 2 окружностей и площади пересечения
    #перенесем систему координат в x1, y1
    #x1 -> 0, y1 -> 0, x2 -> x2 - x1, y2 -> y2 - y1
    x21 = x2 - x1
    y21 = y2 - y1
    
    d = math.sqrt(x21 ** 2 + y21 ** 2)
    res =''
    if x1 == x2 and y1 == y2 and r1 == r2:
        return (math.pi * r1 * r1, "бесконечно много \n общих точек")
    
    if x1 == x2 and y1 == y2 and r1 != r2:
        return (math.pi * min(r1, r2) ** 2, "не пересекаются")
    
    if x1 == x2:
        y = (r1 ** 2 - r2 ** 2 + y21 ** 2) / (2 * y21)
        if r1 ** 2 - y ** 2 < 0:
            if (d > r1 + r2):
                return (0, "не пересекаются")
            return (math.pi * min(r1, r2) ** 2, "не пересекаются")
            
            
        if r1 ** 2 - y ** 2 == 0:
            if (d == r1 + r2):
                return (0, str(x1) + " " + str(y+y1))
            return (math.pi * min(r1, r2) ** 2, str(x1) + " " + str(y+y1))
        
        
        if r1 ** 2 - y ** 2  > 0:                      
            b = (r1 ** 2 - r2 ** 2 + d ** 2) / (2 * d)
            a = (r2 ** 2 - r1 ** 2 + d ** 2) / (2 * d)
            h = math.sqrt(r1 ** 2 - b ** 2)
            sect2 = r2 ** 2 * math.acos(a / r2)
            tria2  = a * h
            sect1 = r1 ** 2 * math.acos(b / r1)
            tria1  = b * h
            s = sect1 + sect2 - tria1 - tria2
            return (s, str(math.sqrt(r1 ** 2 - y ** 2) + x1) + " " + str(y+y1) + "\n"+ str(-math.sqrt(r1 ** 2 - y ** 2) + x1) + " " + str( y + y1))
        
        
    if y1 == y2:
        x = (r1 ** 2 - r2 ** 2 + x21 ** 2) / (2 * x21)
        if r1 ** 2 - x ** 2 < 0:
            if (d > r1 + r2):
                return (0, "не пересекаются")
            return (math.pi * min(r1, r2) ** 2, "не пересекаются")
        
        
        if r1 ** 2 - x ** 2 == 0:
            if (d == r1 + r2):
                return (0, str(x1 + x) + " " + str(y1))
            return (math.pi * min(r1, r2) ** 2, str(x1 + x) + " " + str(y1))
        
        
        if r1 ** 2 - x ** 2  > 0:
            b = (r1 ** 2 - r2 ** 2 + d ** 2) / (2 * d)
            a = (r2 ** 2 - r1 ** 2 + d ** 2) / (2 * d)
            h = math.sqrt(r1 ** 2 - b ** 2)
            sect2 = r2 ** 2 * math.acos(a / r2)
            tria2  = a * h
            sect1 = r1 ** 2 * math.acos(b / r1)
            tria1  = b * h
            s = sect1 + sect2 - tria1 - tria2
            return (s, str(x+x1) + " " + str(math.sqrt(r1 ** 2 - x ** 2) + y1) + "\n"+str(x+x1) + " " + str(-math.sqrt(r1 ** 2 - x ** 2) + y1))
        
        
    if y1 != y2 and x1 != x2:
        c = r1 ** 2 - r2 ** 2 + x21 ** 2 + y21 ** 2
        c_sq = c ** 2
        y21_sq = y21 ** 2
        x21_sq  = x21 ** 2
        di = 16 * y21_sq * c_sq- 16 * (x21_sq + y21_sq) * (c_sq - 4 * r1 * r1 * x21_sq)
        if (di < 0):
            if (d > r1 + r2):
                return (0, "не пересекаются")
            return (math.pi * min(r1, r2) ** 2, "не пересекаются")
        
        
        if (di == 0):
            y = (y21 * c) / (2 * (x21_sq + y21_sq))
            x = (c - 2 * y * y21) / (2 * x21)
            if (d == r1 + r2):
                return (0, str(x1 + x) + " " + str(y1))
            return (math.pi * min(r1, r2) ** 2, str(x1 + x) + " " + str(y1 + y))
        

        if (di > 0):
            y = y_1 = (4 * y21 * c + math.sqrt(di)) / (8 * (x21_sq + y21_sq))
            x = x_1 = (c - 2 * y * y21) / (2 * x21)
            y = y_2 = (4 * y21 * c - math.sqrt(di)) / (8 * (x21_sq + y21_sq))
            x = x_2 = (c - 2 * y * y21) / (2 * x21)
            b = (r1 ** 2 - r2 ** 2 + d ** 2) / (2 * d)
            a = (r2 ** 2 - r1 ** 2 + d ** 2) / (2 * d)
            h = math.sqrt(r1 ** 2 - b ** 2)
            sect2 = r2 ** 2 * math.acos(a / r2)
            tria2  = a * h
            sect1 = r1 ** 2 * math.acos(b / r1)
            tria1  = b * h
            s = sect1 + sect2 - tria1 - tria2
            return (s, str(x_1+x1) + " " + str(y_1+y1) + '\n' + str(x_2 + x1) + " " + str(y2+y_1))



In [18]:
from tkinter import *
import tkinter.scrolledtext as tkst
import numpy as np

class Applic:
    def __init__(self, root):
        self.root = root
        self.set_wind()
        self.nums = []
        self.numsx = []
        self.numsy = []
        self.numb  = -1
        self.not_first = 0
        
        """ Многоугольник """
    
    def perimetr(self):
        self.label_perim['text'] = str(perim(self.nums, len(self.nums)))
        
    def square(self):
        self.label_square['text'] = str(square(self.nums, len(self.nums)))
    
    def is_convex(self):
        if  is_convex(self.nums, len(self.nums)) == False:
            self.label_conv['text'] = "Невыпуклый"
        else:
            self.label_conv['text'] = "Выпуклый"
            
            
    def is_simple(self):
        print(len(self.nums))
        if len(self.nums) > 2:
            p = []
            for i in range(0, len(self.nums) - 1):
                p.append(Seg(self.nums[i], self.nums[i+1], i))

            p.append(Seg(self.nums[0], self.nums[len(self.nums)-1], len(self.nums) - 1))

            if solution(p):
                self.label_simple['text'] = "Не простой"
            else:
                self.label_simple['text'] = "Простой"   
    
            """  if is_simple(self.nums, len(self.nums)):
                self.label_simple['text'] = self.label_simple['text'] + "Простой"
            else:
                self.label_simple['text'] = self.label_simple['text'] + "Не простой" """
        else:
            self.label_simple['text'] = self.label_simple['text'] + "Простой"
                

    def number(self): #ввод числа вершин
        a = self.EntryA.get()
        try:
            a = int(a)
            if (a > 0):
                self.label['text'] = ''
                self.label_nums['text'] = ''
                self.numb = a
            else:
                print("ошибка")
                self.EntryA.delete(0, END) # очищаем текстовое поле полностью
                self.label['text'] = 'Ошибка\n Введите заново'
                self.numb = -1
                #print(self.numb)
        except ValueError:
            print("ошибка")
            self.EntryA.delete(0, END) # очищаем текстовое поле полностью
            self.label['text'] = 'Ошибка\n Введите заново'
            self.numb = -1
            return False
        
    def show(self, event): #ввод многоугольника рисованием
        if self.not_first == 1:
            self.nums.append(Point(self.prev.x, self.prev.y))
            if (dlina(sub(self.first, Point(event.x, event.y))) < 5):
                self.canv.create_line(self.prev.x, self.prev.y, self.first.x, self.first.y) 
            else:
                self.canv.create_line(self.prev.x, self.prev.y, event.x, event.y)
        else:
            self.first = Point(event.x, event.y)
        self.editArea.insert(END,str(event.x) + " " + str(event.y) + "\n") 
        self.prev = Point(event.x, event.y)
        self.not_first  = 1
        
    def clear_all_polygon(self):
        self.canv.delete("all")
        self.editArea.delete(1.0, END)
        self.nums = []
        self.numsx = []
        self.numsy = []
        self.numb  = -1
        self.not_first = 0
        
        self.label['text'] = ''
        self.label_nums['text'] = ''
        self.label_conv['text'] = ""
        self.label_simple['text'] = ""
        self.label_perim['text'] = ""      
        self.label_square['text'] = ""   
        
        self.is_simp = -1
        self.is_conv = -1
        
        
    def polygon(self):
        self.nums = []
        self.numsx = []
        self.numsy = []
        nums = []
        self.canv.delete("all")
        
        lis = []
        a = self.editArea.get(1.0, END)
        lis = a.split()
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums['text'] = 'Ошибка\n Введите заново'

        if len(nums) % 2 != 0 and len(nums) <  2*self.numb:
            self.label_nums['text'] = 'Нечетное число координат\n Введите ещё ' + str(2*self.numb - len(nums))
            return

        elif len(nums) % 2 != 0 and len(nums) >  2*self.numb:
            self.label_nums['text'] = 'Нечетное число координат\n Удалите ' + str(-2*self.numb + len(nums))
            return

        elif self.numb > 0 and len(nums) // 2 < self.numb:
            self.label_nums['text'] = 'Недостаточно вершин\n Введите ещё ' + str(self.numb - len(nums) //2)
            return

        elif self.numb > 0 and len(nums) // 2 > self.numb:
                self.label_nums['text'] = 'Слишком много вершин\n Удалите ' + str(len(nums) //2 - self.numb)
                return

        if (len(nums) > 2):
                for i in range(0, len(nums), 2):
                    self.numsx.append(nums[i])
                    self.numsy.append(nums[i+1])
                    self.nums.append(Point(nums[i], nums[i+1]))
                    

                minx = min(self.numsx)    
                maxx = max(self.numsx)
                if (minx != maxx):
                    kx = 290/(maxx-minx)
                    bx = 35 -kx * minx
                else:
                    kx = 200 / (abs(minx) + 1)
                    bx = 0

                miny = min(self.numsy)
                maxy = max(self.numsy)
                if (miny != maxy):
                    ky = 290/(maxy-miny)
                    by = 5 -ky * miny
                else:
                    ky = 200 / (abs(miny) + 1)
                    by = 0

                for i in range(1, len(self.numsx)):
                    self.canv.create_line(self.numsx[i-1] * kx + bx, self.numsy[i-1] * ky + by,
                                           self.numsx[i] * kx + bx, self.numsy[i]* ky + by)

                self.canv.create_line(self.numsx[0] * kx + bx, self.numsy[0]* ky + by,
                               self.numsx[len(self.numsx)-1] * kx + bx, self.numsy[len(self.numsx)-1]* ky + by) 


    def wind_polygons(self):
            #ввод числа вершин многоугольника
        self.l1 = Label(self.root, text='Число вершин многоугольника',fg = 'blue',
                   font='Arial 16')
        self.l1.grid(row=0, sticky=W)

        self.EntryA = Entry(self.root, width=13, font='Arial 16')
        self.EntryA.grid(row=0, column=1, sticky=E)
        self.label = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label.grid(row=2, column = 1, sticky=E)
        self.but = Button(self.root, width=22, text='Ввести', command = self.number)
        self.but.grid(row=1, column=1, sticky=E)

        self.l2 = Label(self.root, text='Координаты вершин многоугольника',fg = 'blue',
                   font='Arial 16')
        self.l2.grid(row=5, sticky=W)
        #EntryA = Entry(root, width=30, font='Arial 16')
        #EntryA.grid(row=6, column = 0)

        self.editArea = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 20)
        self.editArea.grid(row=6, column = 0, sticky=W)
        self.but2 = Button(self.root, width=22, text='Ввести', command = self.polygon)
        self.but2.grid(row=7, column=0, sticky=W)

        self.but3 = Button(self.root, width=40, text='Проверить на выпуклость', command = self.is_convex)
        self.but3.grid(row=8, column=1, sticky=W)
        self.but5 = Button(self.root, width=40, text='Проверить на самопересечения', command = self.is_simple)
        self.but5.grid(row=10, column=1, sticky=W)
        self.but1 = Button(self.root, width=40, text='Периметр', command = self.perimetr)
        self.but1.grid(row=11, column=1, sticky=W)
        self.but6 = Button(self.root, width=40, text='Площадь простого \n многоугольника', command = self.square)
        self.but6.grid(row=12, column=1, sticky=W)
        self.label_conv = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_conv.grid(row=8, column =2, sticky=W)
        self.label_simple = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_simple.grid(row=10, column =2, sticky=W)
        self.label_perim = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_perim.grid(row=11, column =2, sticky=W)
        self.label_square = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_square.grid(row=12, column =2, sticky=W)
        
        self.but4 = Button(self.root, width=40, text='Очистить всё', command = self.clear_all_polygon)
        self.but4.grid(row=9, column=1, sticky=W)
        self.label_conv = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_conv.grid(row=8, column =2, sticky=W)
        

        self.label_nums = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')

        self.label_nums.grid(row=8, column = 0, sticky=W)
        

        self.canv = Canvas(self.root, width=400, height=300, bg="white")
        self.canv.grid(row=6, column=1, columnspan=7,
                               padx=5, pady=5, sticky=E+W+S+N)  
        self.canv.bind("<1>", self.show)         
        
    def wind_polygons_close(self):
        self.clear_all_polygon()
        self.canv.grid_remove()
        self.label_nums.grid_remove()
        self.label_simple.grid_remove()
        self.label_perim.grid_remove()
        self.label_square.grid_remove()        
        self.label_conv.grid_remove()
        self.l1.grid_remove()
        self.l2.grid_remove()
        
        
        self.but.grid_remove()
        self.but1.grid_remove()        
        self.but2.grid_remove()
        self.but3.grid_remove()
        self.but4.grid_remove()
        self.but5.grid_remove()        
        self.but6.grid_remove()     
        
        self.editArea.grid_remove()        
        self.EntryA.grid_remove() 
        self.label.grid_remove() 
        
        
        """ПРЯМОУГОЛЬНИКИ"""
   


    def clear_all_rect(self):
        self.canv.delete("all")
        self.editArea1.delete(1.0, END)
        self.editArea2.delete(1.0, END)
        self.rect_num = 0
        self.label_square['text'] = ''
        
            
    def wind_rect_close(self):
        self.canv.grid_remove()
        self.label_square.grid_remove()     
        self.l1.grid_remove()
        self.l2.grid_remove()
        self.l3.grid_remove()        
        self.label_nums1.grid_remove()      
        
        self.but2.grid_remove()
        self.but1.grid_remove()  
        self.but4.grid_remove()
        
        self.editArea1.grid_remove()        
        self.editArea2.grid_remove() 
        self.rect_num = 0
       
        
    def get_rect_coord(self):
        nums = []
        self.canv.delete("all")
        lis = []
        a = self.editArea1.get(1.0, END)
        lis = a.split()
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea1.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'
                return

        if len(nums) < 4:
            self.label_nums1['text'] = 'Слишком мало координат для 1\n Введите ещё ' + str(4 - len(nums))
            return

        elif len(nums) > 4:
                self.label_nums1['text'] = 'Слишком много вершин для 1\n Удалите ' + str(len(nums) - 4)
                return

        x1_min = self.x1_min = min(nums[0], nums[2])
        y1_min = self.y1_min = min(nums[1], nums[3])
        x1_max = self.x1_max = max(nums[0], nums[2])                    
        y1_max = self.y1_max = max(nums[1], nums[3])
        
        a = self.editArea2.get(1.0, END)
        lis = a.split()
        nums = []
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea2.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'
                return

        if len(nums) < 4:
            self.label_nums1['text'] = 'Слишком мало координат для 2\n Введите ещё ' + str(4 - len(nums))
            return

        elif len(nums) > 4:
                self.label_nums1['text'] = 'Слишком много вершин для 2\n Удалите ' + str(len(nums) - 4)
                return

        x2_min = self.x2_min = min(nums[0], nums[2])
        y2_min = self.y2_min = min(nums[1], nums[3])
        x2_max = self.x2_max = max(nums[0], nums[2])                    
        y2_max = self.y2_max = max(nums[1], nums[3])
        
        
        minx = min(x1_min, x2_min, y1_min, y2_min)    
        maxx = max(x1_max, x2_max, y1_max, y2_max)
        if (minx != maxx):
            ky = kx = 290/(maxx-minx)
            by = bx = 35 -kx * minx
        else:
            ky = kx = 200 / (abs(minx) + 1)
            by = bx = 0

        self.canv.create_line(x1_min * kx + bx, y1_min * ky + by,x1_max * kx + bx, y1_min * ky + by)
        self.canv.create_line(x1_max * kx + bx, y1_min * ky + by,x1_max * kx + bx, y1_max * ky + by)                                         
        self.canv.create_line(x1_max * kx + bx, y1_max * ky + by,x1_min * kx + bx, y1_max * ky + by)                             
        self.canv.create_line(x1_min * kx + bx, y1_max * ky + by,x1_min * kx + bx, y1_min * ky + by)
        
        self.canv.create_line(x2_min * kx + bx, y2_min * ky + by,x2_max * kx + bx, y2_min * ky + by)
        self.canv.create_line(x2_max * kx + bx, y2_min * ky + by,x2_max * kx + bx, y2_max * ky + by)                                         
        self.canv.create_line(x2_max * kx + bx, y2_max * ky + by,x2_min * kx + bx, y2_max * ky + by)                             
        self.canv.create_line(x2_min * kx + bx, y2_max * ky + by,x2_min * kx + bx, y2_min * ky + by)
        
    def rect_intersect(self):
        if self.x1_min > self.x2_max or  self.x2_min > self.x1_max or self.y1_min > self.y2_max or self.y2_min > self.y1_max:
            self.label_square['text'] = "Фигуры \n Не пересекаются"
        else:
            self.label_square['text'] = str((min(self.x1_max, self.x2_max) - max(self.x1_min, self.x2_min)) * (min(self.y1_max, self.y2_max) - max(self.y1_min, self.y2_min)))
        
    def show_rect(self, event): #ввод многоугольника рисованием
        
        self.rect_num += 1
        if self.rect_num == 1 or self.rect_num == 3:
            self.rect_x = event.x
            self.rect_y = event.y
            if self.rect_num == 1:
                self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            else:
                self.editArea2.insert(END,str(event.x) + " " + str(event.y) + "\n") 
        if self.rect_num == 2 or self.rect_num == 4:
            self.canv.create_line(self.rect_x, self.rect_y, event.x, self.rect_y) 
            self.canv.create_line(event.x, self.rect_y, event.x, event.y)                 
            self.canv.create_line(event.x, event.y, self.rect_x, event.y) 
            self.canv.create_line(self.rect_x, event.y, self.rect_x, self.rect_y)     
            if self.rect_num == 2:
                self.x2_min = min(self.rect_x, event.x)
                self.x2_max = max(self.rect_x, event.x)            
                self.y2_min = min(self.rect_y, event.y)            
                self.y2_max = max(self.rect_y, event.y)               
                self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            else:
                self.x1_min = min(self.rect_x, event.x)
                self.x1_max = max(self.rect_x, event.x)            
                self.y1_min = min(self.rect_y, event.y)            
                self.y1_max = max(self.rect_y, event.y)  
                self.editArea2.insert(END,str(event.x) + " " + str(event.y) + "\n")       
        if self.rect_num == 4:
            self.rect_num  = 0

    def wind_rect(self):
        self.rect_num = 0
        self.rect_x = 0
        self.rect_y = 0
        self.l1 = Label(self.root, text='Задайте прямоугольники через \n координаты противоположных вершин',fg = 'gray',
                   font='Arial 16')
        self.l1.grid(row=0, sticky=W)


        self.l2 = Label(self.root, text='Первый прямоугольник',fg = 'blue',
                   font='Arial 16')
        self.l2.grid(row=5, sticky=W)
        #EntryA = Entry(root, width=30, font='Arial 16')
        #EntryA.grid(row=6, column = 0)

        self.editArea1 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea1.grid(row=6, column = 0, sticky=W)
        self.l3 = Label(self.root, text='Второй прямоугольник',fg = 'blue',
                   font='Arial 16')
        self.l3.grid(row=7, sticky=W)
        self.editArea2 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea2.grid(row=8, column = 0, sticky=W)

        
        self.but2 = Button(self.root, width=22, text='Ввести', command = self.get_rect_coord)
        self.but2.grid(row=9, column=0, sticky=W)
        
        self.canv = Canvas(self.root, width=400, height=350, bg="white")
        self.canv.grid(row=6, column=1, columnspan=7, rowspan = 3,
                               padx=5, pady=5, sticky=E)  
        self.canv.bind("<1>", self.show_rect)  
        
        self.label_nums1 = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_nums1.grid(row=10, column = 0, sticky=E)

        self.but1 = Button(self.root, width=40, text='Площадь пересечения', command = self.rect_intersect)
        self.but1.grid(row=12, column=0, sticky=W)
 
        self.label_square = Label(self.root, text='' ,fg = 'Gray',
                   font='Arial 16')
        self.label_square.grid(row=12, column = 1, sticky=W)
        self.but4 = Button(self.root, width=40, text='Очистить всё', command = self.clear_all_rect)
        self.but4.grid(row=10, column=1, sticky=W)
        
        """ТРЕУГОЛЬНИК"""

    def clear_all_tria(self):
        self.canv.delete("all")
        self.editArea1.delete(1.0, END)
        self.editArea2.delete(1.0, END)
        self.tria_num = 0
        self.label_square['text'] = ''
        
        
    def get_tria_coord(self):
        nums = []
        self.canv.delete("all")
        lis = []
        a = self.editArea1.get(1.0, END)
        lis = a.split()
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea1.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'
                return

        if len(nums) < 6:
            self.label_nums1['text'] = 'Слишком мало координат для 1\n Введите ещё ' + str(6 - len(nums))
            return

        elif len(nums) > 6:
                self.label_nums1['text'] = 'Слишком много вершин для 1\n Удалите ' + str(len(nums) - 6)
                return
        
        self.a = Point(nums[0], nums[1])
        self.b = Point(nums[2], nums[3])
        self.c = Point(nums[4], nums[5])
        
        a = self.editArea2.get(1.0, END)
        lis = a.split()
        nums = []
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea2.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'
                return

        if len(nums) < 4:
            self.label_nums1['text'] = 'Слишком мало координат для 2\n Введите ещё ' + str(4 - len(nums))
            return

        elif len(nums) > 4:
                self.label_nums1['text'] = 'Слишком много вершин для 2\n Удалите ' + str(len(nums) - 4)
                return
            
        self.otr1 = Point(nums[0], nums[1])
        self.otr2 = Point(nums[2], nums[3])
        
        minx = min(self.a.x, self.b.x, self.c.x, self.otr1.x, self.otr2.x)   
        minx = min(minx, self.a.y, self.b.y, self.c.y, self.otr1.y, self.otr2.y)
        
        maxx = max(self.a.x, self.b.x, self.c.x, self.otr1.x, self.otr2.x)   
        maxx = max(maxx, self.a.y, self.b.y, self.c.y, self.otr1.y, self.otr2.y)
                
        if (minx != maxx):
            ky = kx = 290/(maxx-minx)
            by = bx = 35 -kx * minx
        else:
            ky = kx = 200 / (abs(minx) + 1)
            by = bx = 0

        self.canv.create_line(self.a.x * kx + bx, self.a.y * ky + by, self.b.x * kx + bx, self.b.y * ky + by)
        self.canv.create_line(self.a.x * kx + bx, self.a.y * ky + by, self.c.x * kx + bx, self.c.y * ky + by)                                         
        self.canv.create_line(self.c.x * kx + bx, self.c.y * ky + by, self.b.x * kx + bx, self.b.y * ky + by)                             
       
        self.canv.create_line(self.otr1.x* kx + bx, self.otr1.y * ky + by, self.otr2.x * kx + bx, self.otr2.y * ky + by)
        
    def show_tria(self, event):
        self.tria_num += 1
        if self.tria_num == 1:
            self.a = Point(event.x, event.y)
            self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            
        if self.tria_num == 4:
            self.otr1 = Point(event.x, event.y)
            self.editArea2.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            
        if self.tria_num == 2:
            self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            self.canv.create_line(self.a.x, self.a.y, event.x, event.y) 
            self.b = Point(event.x, event.y)
        
        if self.tria_num == 3:
            self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            self.canv.create_line(self.b.x, self.b.y, event.x, event.y) 
            self.canv.create_line(self.a.x, self.a.y, event.x, event.y)             
            self.c = Point(event.x, event.y)
            
        if self.tria_num == 5:
            self.otr2 = Point(event.x, event.y)    
            self.editArea2.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            self.canv.create_line(self.otr1.x, self.otr1.y, self.otr2.x, self.otr2.y)
            self.tria_num  = 0
    
    def tria_intersect(self):
        res = tria(self.a, self.b, self.c, self.otr1, self.otr2)
        if res == 1000:
            self.label_square['text'] = "Бесконечно много общих точек"
        else:
            self.label_square['text'] = str(res)
     
    def wind_tria_close(self):
        self.canv.grid_remove()
        self.label_square.grid_remove()     
        self.l1.grid_remove()
        self.l2.grid_remove()
        self.l3.grid_remove()        
        self.label_nums1.grid_remove()      
        
        self.but2.grid_remove()
        self.but1.grid_remove()  
        self.but4.grid_remove()
        
        self.editArea1.grid_remove()        
        self.editArea2.grid_remove() 
        self.tria_num = 0
            
    
        
    def wind_tria(self):
        self.tria_num = 0
        self.a = 0
        self.b = 0
        self.c = 0
        self.l1 = Label(self.root, text='Задайте координаты вершин \n треугольника и отрезка',fg = 'gray',
                   font='Arial 16')
        self.l1.grid(row=0, sticky=W)


        self.l2 = Label(self.root, text='Треугольник',fg = 'blue',
                   font='Arial 16')
        self.l2.grid(row=5, sticky=W)
        #EntryA = Entry(root, width=30, font='Arial 16')
        #EntryA.grid(row=6, column = 0)

        self.editArea1 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea1.grid(row=6, column = 0, sticky=W)
        self.l3 = Label(self.root, text='Отрезок',fg = 'blue',
                   font='Arial 16')
        self.l3.grid(row=7, sticky=W)
        self.editArea2 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea2.grid(row=8, column = 0, sticky=W)
        
        self.but2 = Button(self.root, width=22, text='Ввести', command = self.get_tria_coord)
        self.but2.grid(row=9, column=0, sticky=W)
        
        self.canv = Canvas(self.root, width=400, height=350, bg="white")
        self.canv.grid(row=6, column=1, columnspan=7, rowspan = 3,
                               padx=5, pady=5, sticky=E)  
        self.canv.bind("<1>", self.show_tria)  
        
        self.label_nums1 = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_nums1.grid(row=10, column = 0, sticky=E)

        self.but1 = Button(self.root, text='Количество точек пересечения', command = self.tria_intersect)
        self.but1.grid(row=12, column=0, sticky=W)
 
        self.label_square = Label(self.root, text='' ,fg = 'Gray',
                   font='Arial 16')
        self.label_square.grid(row=12, column = 1, sticky=W)
        self.but4 = Button(self.root, width=40, text='Очистить всё', command = self.clear_all_tria)
        self.but4.grid(row=10, column=1, sticky=W)
        
        """КРУГИ"""

    def clear_all_circle(self):
        self.canv.delete("all")
        self.editArea1.delete(1.0, END)
        self.editArea2.delete(1.0, END)
        self.circle_num = 0
        self.label_square['text'] = ''
        self.label_points['text'] = ''
        
        
    def wind_circle_close(self):
        self.canv.grid_remove()
        self.label_square.grid_remove()  
        self.label_points.grid_remove()  
        self.l1.grid_remove()
        self.l2.grid_remove()
        self.l3.grid_remove()        
        self.label_nums1.grid_remove()      
        
        self.but2.grid_remove()
        self.but1.grid_remove()  
        self.but4.grid_remove()
        self.but3.grid_remove()
        
        self.editArea1.grid_remove()        
        self.editArea2.grid_remove() 
        self.circle_num = 0
        
    
             
    def get_circle_coord(self):
        nums = []
        self.canv.delete("all")
        lis = []
        a = self.editArea1.get(1.0, END)
        lis = a.split()
        for iter in lis:
            try:
                val = float(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea1.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'
                return

        if len(nums) < 3:
            self.label_nums1['text'] = 'Слишком мало данных для 1\n Введите ещё ' + str(3 - len(nums))
            return

        elif len(nums) > 3:
                self.label_nums1['text'] = 'Слишком много данных для 1\n Удалите ' + str(len(nums) - 3)
                return
        
        x1 = self.x1 = nums[0]
        y1  = self.y1 = nums[1]
        r1  = self.r1 = nums[2]
        if nums[2] <= 0:
            self.label_nums1['text'] = 'Ошибка\n Радиус положительный!'
            self.editArea1.delete(1.0, END) # очищаем текстовое поле полностью
            return 
        
        a = self.editArea2.get(1.0, END)
        lis = a.split()
        nums = []
        for iter in lis:
            try:
                val = float(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea2.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'
                return

        if len(nums) < 3:
            self.label_nums1['text'] = 'Слишком мало данных для 2\n Введите ещё ' + str(3 - len(nums))
            return

        elif len(nums) > 3:
                self.label_nums1['text'] = 'Слишком много данных для 2\n Удалите ' + str(len(nums) - 3)
                return
            
        x2 = self.x2 = nums[0]
        y2 = self.y2 = nums[1]
        r2 = self.r2 = nums[2]
        if nums[2] <= 0:
            self.label_nums1['text'] = 'Ошибка\n Радиус положительный!'
            self.editArea2.delete(1.0, END) # очищаем текстовое поле полностью
            return 
        
        minx = min(x1 - r1, x2 - r2)   
        maxx = max(x1 + r1, x2 + r2)  
        miny = min(y1 - r1, y2 - r2)   
        maxy = max(y1 + r1, y2 + r2)     
                
      
        self.canv.create_oval(x1 - r1, y1 - r1, x1 + r1, y1 + r1)
        self.canv.create_oval(x2 - r2, y2 - r2, x2 + r2, y2 + r2)
        
    def show_circle(self, event):
        self.circle_num += 1
        if self.circle_num == 1:
            self.x1 = event.x
            self.y1 = event.y
            self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            
        if self.circle_num == 3:
            self.x2 = event.x
            self.y2 = event.y
            self.editArea2.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            
        if self.circle_num == 2:
            r = math.sqrt((self.x1 -event.x) ** 2 + (self.y1 -event.y) ** 2 )
            self.r1 = r
            self.editArea1.insert(END,str(r))
            self.canv.create_oval(self.x1 - self.r1, self.y1 - self.r1, self.x1 + self.r1, self.y1 + self.r1)
        
        if self.circle_num == 4:
            r = math.sqrt((self.x2 -event.x) ** 2 + (self.y2 -event.y) ** 2 )
            self.r2 = r
            self.editArea2.insert(END,str(r))
            self.canv.create_oval(self.x2 - self.r2, self.y2 - self.r2, self.x2 + self.r2, self.y2 + self.r2)
            self.circle_num  = 0
    
    def circle_intersect(self):
        res = circles(self.x1, self.y1, self.r1, self.x2, self.y2, self.r2)
        self.label_square['text'] = str(res[0])
        
    def circle_points(self):
        res = circles(self.x1, self.y1, self.r1, self.x2, self.y2, self.r2)
        self.label_points['text'] = res[1]
    
        
    def wind_circle(self):
        self.circle_num = 0
        self.x1 = 0
        self.x2 = 0
        self.y1 = 0
        self.y2 = 0
        self.r1 = 0
        self.r2 = 0
        self.l1 = Label(self.root, text='Задайте координаты центра и радиусы',fg = 'gray',
                   font='Arial 16')
        self.l1.grid(row=0, sticky=W)


        self.l2 = Label(self.root, text='Первый круг',fg = 'blue',
                   font='Arial 16')
        self.l2.grid(row=5, sticky=W)
        #EntryA = Entry(root, width=30, font='Arial 16')
        #EntryA.grid(row=6, column = 0)

        self.editArea1 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea1.grid(row=6, column = 0, sticky=W)
        self.l3 = Label(self.root, text='Второй круг',fg = 'blue',
                   font='Arial 16')
        self.l3.grid(row=7, sticky=W)
        self.editArea2 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea2.grid(row=8, column = 0, sticky=W)
        
        self.but2 = Button(self.root, width=22, text='Ввести', command = self.get_circle_coord)
        self.but2.grid(row=9, column=0, sticky=W)
        
        self.canv = Canvas(self.root, width=400, height=350, bg="white")
        self.canv.grid(row=6, column=1, columnspan=7, rowspan = 3,
                               padx=5, pady=5, sticky=E)  
        self.canv.bind("<1>", self.show_circle)  
        
        self.label_nums1 = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_nums1.grid(row=10, column = 0, sticky=E)

        self.but1 = Button(self.root, text='Площадь пересечения', command = self.circle_intersect)
        self.but1.grid(row=12, column=0, sticky=W)
        self.label_square = Label(self.root, text='' ,fg = 'Gray',
                   font='Arial 16')
        self.label_square.grid(row=12, column = 1, sticky=W)
        
        self.but3 = Button(self.root, text='Точки пересечения', command = self.circle_points)
        self.but3.grid(row=13, column=0, sticky=W)
        self.label_points = Label(self.root, text='' ,fg = 'Gray',
                   font='Arial 16')
        self.label_points.grid(row=13, column = 1, sticky=W)
        
        self.but4 = Button(self.root, width=40, text='Очистить всё', command = self.clear_all_circle)
        self.but4.grid(row=10, column=1, sticky=W)
        
        """МНОГОУГОЛЬНИКИ"""
        
    def wind_polys_close(self):
        self.canv.grid_remove() 
        self.label_nums1.grid_remove()  
        self.label_simple1.grid_remove()  
        self.label_simple2.grid_remove()  
        self.label_intersect.grid_remove()  
        self.l1.grid_remove()
        self.l2.grid_remove()
        self.l3.grid_remove()        
        self.label_nums1.grid_remove()      
        self.label_polys.grid_remove()
        
        self.but2.grid_remove()
        self.but1.grid_remove()  
        self.but4.grid_remove()
        self.but5.grid_remove()
        self.but6.grid_remove()
        self.but7.grid_remove()
        
        self.editArea1.grid_remove()        
        self.editArea2.grid_remove() 
        self.nums1 = []
        self.nums2 = []
        self.polys = 1
        self.polys_not_first = 0
        self.polys_prev = 0
        self.polys_first = 0
            

    def enter_polys(self):
        self.nums1 = []
        self.numsx1 = []
        self.numsy1 = []
        nums = []
        self.canv.delete("all")
        
        lis = []
        a = self.editArea1.get(1.0, END)
        lis = a.split()
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea1.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'

        if len(nums) % 2 != 0:
            self.label_nums1['text'] = 'Нечетное число координат\n Введите ещё ' + str(2*self.numb - len(nums))
            return

        if (len(nums) > 2):
                for i in range(0, len(nums), 2):
                    self.numsx1.append(nums[i])
                    self.numsy1.append(nums[i+1])
                    self.nums1.append(Point(nums[i], nums[i+1]))
                    
        self.nums2 = []
        self.numsx2 = []
        self.numsy2 = []
        nums = []
        
        lis = []
        a = self.editArea2.get(1.0, END)
        lis = a.split()
        for iter in lis:
            try:
                val = int(iter)
                self.label_nums1['text'] = ''
                nums.append(val)
            except ValueError:
                print("ошибка")
                self.editArea2.delete(1.0, END) # очищаем текстовое поле полностью
                self.label_nums1['text'] = 'Ошибка\n Введите заново'

        if len(nums) % 2 != 0:
            self.label_nums1['text'] = 'Нечетное число координат\n Введите ещё ' + str(2*self.numb - len(nums))
            return

        if (len(nums) > 2):
                for i in range(0, len(nums), 2):
                    self.numsx2.append(nums[i])
                    self.numsy2.append(nums[i+1])
                    self.nums2.append(Point(nums[i], nums[i+1]))
                    

        minx = min(min(self.numsx1), min(self.numsx2))
        maxx = max(max(self.numsx1), max(self.numsx2))
        miny = min(min(self.numsy1), min(self.numsy2))
        maxy = max(max(self.numsy1), max(self.numsy2))
        if (minx != maxx):
            kx = 290/(maxx-minx)
            bx = 35 -kx * minx
        else:
            kx = 200 / (abs(minx) + 1)
            bx = 0
        if (miny != maxy):
            ky = 290/(maxy-miny)
            by = 5 -ky * miny
        else:
            ky = 200 / (abs(miny) + 1)
            by = 0

        print("g")
            
        if len(self.numsx1) > 1:
            for i in range(1, len(self.numsx1)):
                self.canv.create_line(self.numsx1[i-1] * kx + bx, self.numsy1[i-1] * ky + by,
                                       self.numsx1[i] * kx + bx, self.numsy1[i]* ky + by)

            self.canv.create_line(self.numsx1[0] * kx + bx, self.numsy1[0]* ky + by,
                           self.numsx1[len(self.numsx1)-1] * kx + bx, self.numsy1[len(self.numsx1)-1]* ky + by)
        
        if len(self.numsx2) > 1:
            for i in range(1, len(self.numsx2)):
                self.canv.create_line(self.numsx2[i-1] * kx + bx, self.numsy2[i-1] * ky + by,
                                   self.numsx2[i] * kx + bx, self.numsy2[i]* ky + by, fill = "red")

            self.canv.create_line(self.numsx2[0] * kx + bx, self.numsy2[0]* ky + by,
                       self.numsx2[len(self.numsx2)-1] * kx + bx, self.numsy2[len(self.numsx2)-1]* ky + by, fill = "red")
    
    def first_poly(self):
        self.polys = 2
        self.polys_not_first = 0
        self.polys_prev = 0
        self.polys_first = 0
        
    def second_poly(self):
        self.polys = 1
        
        self.polys_not_first = 0
        self.polys_prev = 0
        self.polys_first = 0  
        
    
    def show_polys(self, event): #ввод многоугольника рисованием
        if self.polys == 1:
            if self.polys_not_first == 1:
                self.nums1.append(Point(self.polys_prev.x, self.polys_prev.y))
                if (dlina(sub(self.polys_first, Point(event.x, event.y))) < 5):
                    self.canv.create_line(self.polys_prev.x, self.polys_prev.y, self.polys_first.x, self.polys_first.y) 
                else:
                    self.canv.create_line(self.polys_prev.x, self.polys_prev.y, event.x, event.y)
            else:
                self.polys_first = Point(event.x, event.y)
            self.editArea1.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            self.polys_prev = Point(event.x, event.y)
            self.polys_not_first  = 1
            
        if self.polys == 2:
            if self.polys_not_first == 1:
                self.nums2.append(Point(self.polys_prev.x, self.polys_prev.y))
                if (dlina(sub(self.polys_first, Point(event.x, event.y))) < 5):
                    self.canv.create_line(self.polys_prev.x, self.polys_prev.y, self.polys_first.x, self.polys_first.y, fill = 'red') 
                else:
                    self.canv.create_line(self.polys_prev.x, self.polys_prev.y, event.x, event.y, fill = 'red')
            else:
                self.polys_first = Point(event.x, event.y)
            self.editArea2.insert(END,str(event.x) + " " + str(event.y) + "\n") 
            self.polys_prev = Point(event.x, event.y)
            self.polys_not_first  = 1
        
    def clear_all_polys(self):
        self.polys = 1
        
        self.polys_not_first = 0
        self.polys_prev = 0
        self.polys_first = 0  
        
        self.canv.delete("all")
        self.editArea1.delete(1.0, END)
        self.editArea2.delete(1.0, END)

        self.nums1 = []
        self.nums2 = []
        self.label_nums1['text'] = ''
        self.label_simple1['text'] = ""
        self.label_simple2['text'] = ""
        self.label_intersect['text'] = ""
        self.label_polys['text'] = ""
    
    def polys_intersect(self):
        simple1 = True
        if len(self.nums1) > 2:
            q = []
            for i in range(0, len(self.nums1) - 1):
                q.append(Seg(self.nums1[i], self.nums1[i+1], i))

            q.append(Seg(self.nums1[0], self.nums1[len(self.nums1)-1], len(self.nums1) - 1))

            simple1 =  not solution(q)
        if not simple1:
            self.label_simple1['text'] = "Не простой"
        else:
            self.label_simple1['text'] = "Простой"   
            
        simple2 = True
        if len(self.nums2) > 2:
            p = []
            for i in range(0, len(self.nums2) - 1):
                p.append(Seg(self.nums2[i], self.nums2[i+1], i))

            p.append(Seg(self.nums2[0], self.nums2[len(self.nums2)-1], len(self.nums2) - 1))

            simple2 =  not solution(p)
        if not simple2:
            self.label_simple2['text'] = "Не простой"
        else:
            self.label_simple2['text'] = "Простой"   

        ans = False    
                
        if not simple1 or not simple2:
            for i in range(len(self.nums1) - 1):
                for j  in range(len(self.nums2) - 1):
                    if segments_cross(self.nums1[i], self.nums1[i + 1], self.nums2[j], self.nums2[j+1]):
                        ans = True
                        break
                if len(self.nums2) >= 1 and segments_cross(self.nums1[i], self.nums1[i + 1], self.nums2[0], self.nums2[len(self.nums2) - 1]):
                    ans = True
                    break
                    
            for j  in range(len(self.nums2) - 1):
                if segments_cross(self.nums1[0], self.nums1[len(self.nums1) - 1], self.nums2[j], self.nums2[j+1]):
                    ans= True
                    break
            if len(self.nums1) >= 1 and len(self.nums2) - 1 and segments_cross(self.nums1[0], self.nums1[len(self.nums1) - 1], self.nums2[0], self.nums2[len(self.nums2) - 1]):
                ans = True
            
            if not ans:
                self.label_intersect['text'] = "Не пересекаются"
                return False
            else:
                self.label_intersect['text'] = "Пересекаются" 
                return True
            return
        
        if len(self.nums2) > 2:
            p = []
            for i in range(0, len(self.nums2) - 1):
                p.append(Seg(self.nums2[i], self.nums2[i+1], i + len(self.nums1)))

            p.append(Seg(self.nums2[0], self.nums2[len(self.nums2)-1], len(self.nums2) - 1 + len(self.nums1)))

        ans = solution_polys(q, p)
        if not ans:
                self.label_intersect['text'] = "Не пересекаются"
                return False
        else:
                self.label_intersect['text'] = "Пересекаются"
                return True
        print(ans)
                
            
    def poly_in_poly(self):
        simple1 = True
        if len(self.nums1) > 2:
            q = []
            for i in range(0, len(self.nums1) - 1):
                q.append(Seg(self.nums1[i], self.nums1[i+1], i))

            q.append(Seg(self.nums1[0], self.nums1[len(self.nums1)-1], len(self.nums1) - 1))

            simple1 =  not solution(q)
            
        simple2 = True
        if len(self.nums2) > 2:
            p = []
            for i in range(0, len(self.nums2) - 1):
                p.append(Seg(self.nums2[i], self.nums2[i+1], i))

            p.append(Seg(self.nums2[0], self.nums2[len(self.nums2)-1], len(self.nums2) - 1))

            simple2 =  not solution(p)
        nums1 = self.nums1
        nums2 = self.nums2
        print(simple1, simple2)
        if not simple1 or not simple2:
            self.label_polys['text'] = "Не простые"
            return
        simple = self.polys_intersect()
        print(simple)
        
        p1 = point_in_poly(nums1, nums2[0])
        p2 = point_in_poly(nums2, nums1[0])
        
        if simple:
            self.label_polys['text'] = "Нет"
            return
        if not simple and  simple1 and simple2 and (p1 or p2):
            print(p1 or p2)
            self.label_polys['text'] = "Да"
            return
        if not simple and  simple1 and simple2 and not p1 and not p2:
            self.label_polys['text'] = "Нет"     
            return
        
    def wind_polys(self):
        self.polys = 1
        self.polys_num = 0
        self.nums1 = []
        self.nums2 = []
        self.polys_not_first = 0
        self.polys_prev = 0
        self.polys_first = 0  
        self.x1 = 0
        self.x2 = 0
        self.y1 = 0
        self.y2 = 0
        self.r1 = 0
        self.r2 = 0
        self.l1 = Label(self.root, text='Задайте координаты вершин \n многоугольников',fg = 'gray',
                   font='Arial 16')
        self.l1.grid(row=0, sticky=W)


        self.l2 = Label(self.root, text='Первый многоугольник',fg = 'blue',
                   font='Arial 16')
        self.l2.grid(row=5, sticky=W)
        #EntryA = Entry(root, width=30, font='Arial 16')
        #EntryA.grid(row=6, column = 0)

        self.editArea1 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea1.grid(row=6, column = 0, sticky=W)
        self.l3 = Label(self.root, text='Второй многоугольник',fg = 'blue',
                   font='Arial 16')
        self.l3.grid(row=7, sticky=W)
        self.editArea2 = tkst.ScrolledText(self.root, wrap   = WORD, width  = 20, height = 10)
        self.editArea2.grid(row=8, column = 0, sticky=W)
        
        self.but2 = Button(self.root, width=22, text='Ввести', command = self.enter_polys)
        self.but2.grid(row=9, column=0, sticky=W)
        
        self.canv = Canvas(self.root, width=400, height=350, bg="white")
        self.canv.grid(row=6, column=1, columnspan=7, rowspan = 3,
                               padx=5, pady=5, sticky=E)  
        self.canv.bind("<1>", self.show_polys)  
        
        self.label_nums1 = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_nums1.grid(row=10, column = 0, sticky=E)

        self.but1 = Button(self.root, text='Проверить на пересечения', command = self.polys_intersect)
        self.but1.grid(row=12, column=0, sticky=W)
        self.label_square = Label(self.root, text='' ,fg = 'Gray',
                   font='Arial 16')
        self.label_square.grid(row=12, column = 1, sticky=W)
        
           
        self.but4 = Button(self.root, width=40, text='Очистить всё', command = self.clear_all_polys)
        self.but4.grid(row=12, column=1, sticky=W)
        
        self.but5 = Button(self.root, width=40, text='Ввести 1', command = self.first_poly)
        self.but5.grid(row=10, column=1, sticky=W)
        self.label_simple1 = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_simple1.grid(row=10, column = 2, sticky=E)
        self.label_simple2 = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_simple2.grid(row=11, column = 2, sticky=E)
        
        self.label_intersect = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_intersect.grid(row=12, column = 2, sticky=E)
        
        self.but6 = Button(self.root, width=40, text='Ввести 2', command = self.second_poly)
        self.but6.grid(row=11, column=1, sticky=W)
        self.but7 = Button(self.root, text='Лежит ли многоугольник \n строго внутри другом \n (для простых)', command = self.poly_in_poly)
        self.but7.grid(row=13, column=0, sticky=W)
        self.label_polys = Label(self.root, text='',fg = 'Gray',
                   font='Arial 16')
        self.label_polys.grid(row=13, column = 2, sticky=E)
        
        """Окно приложения"""
        

    def set_wind(self):
        mainmenu = Menu(self.root) 
        self.root.config(menu=mainmenu) 

        polygon = Menu(mainmenu, tearoff=0)
        polygon.add_command(label="Открыть...", command = self.wind_polygons)
        polygon.add_command(label="Закрыть...", command = self.wind_polygons_close)
        
        rect = Menu(mainmenu, tearoff=0)
        rect.add_command(label="Открыть...", command = self.wind_rect)
        rect.add_command(label="Закрыть...", command = self.wind_rect_close)
        
        polys = Menu(mainmenu, tearoff=0)
        polys.add_command(label="Открыть...", command = self.wind_polys)
        polys.add_command(label="Закрыть...", command = self.wind_polys_close)
        
        circle = Menu(mainmenu, tearoff=0)
        circle.add_command(label="Открыть...", command = self.wind_circle)
        circle.add_command(label="Закрыть...", command = self.wind_circle_close)
 
        triangle = Menu(mainmenu, tearoff=0)
        triangle.add_command(label="Открыть...", command = self.wind_tria)
        triangle.add_command(label="Закрыть...", command = self.wind_tria_close)
    
        mainmenu.add_cascade(label="Многоугольник", menu=polygon, font='Arial 18')
        mainmenu.add_cascade(label="Прямоугольники", menu=rect, font='Arial 18')
        mainmenu.add_cascade(label="Треугольник", menu=triangle, font='Arial 18')
        mainmenu.add_cascade(label="Круги", menu=circle, font='Arial 18')
        mainmenu.add_cascade(label="Многоугольники", menu=polys, font='Arial 18')


In [ ]:
def main():
    root = Tk()
    root.title('Многоугольники')
    root.geometry('900x700')
    app = Applic(root)
    root.mainloop()
    
if __name__ == '__main__':
    main()


11
